## Drunk people

_setup_

In [195]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [196]:
import pandas as pd
import numpy as np

from pandas.plotting import scatter_matrix
from seaborn import scatterplot, heatmap

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.preprocessing import OneHotEncoder

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.metrics import mean_absolute_error

from joblib import dump

_read dataset_

In [197]:
# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv("./ai1/datasets/dataset_alcohol.csv")

# Shuffle the dataset
df = df.sample(frac=1, random_state=2)
df.reset_index(drop=True, inplace=True)

In [198]:
df.shape

(76, 9)

In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age_yrs        76 non-null     int64  
 1   height_cm      76 non-null     int64  
 2   weight_kg      76 non-null     int64  
 3   duration_mins  76 non-null     object 
 4   elapsed_mins   76 non-null     object 
 5   sex            76 non-null     object 
 6   last_meal      65 non-null     object 
 7   units          76 non-null     float64
 8   over_limit     76 non-null     object 
dtypes: float64(1), int64(3), object(5)
memory usage: 5.5+ KB


In [200]:
df.describe(include = "all")

,age_yrs,height_cm,weight_kg,duration_mins,elapsed_mins,sex,last_meal,units,over_limit
count,76.000000,76.000000,76.000000,76,76,76,65,76.000000,76
unique,NaN,NaN,NaN,16,9,2,4,NaN,2
top,NaN,NaN,NaN,120,10,Male,Full,NaN,No
freq,NaN,NaN,NaN,15,61,60,33,NaN,46
mean,22.657895,176.644737,71.486842,NaN,NaN,NaN,NaN,8.632895,NaN
std,5.627439,8.453329,11.474602,NaN,NaN,NaN,NaN,5.775567,NaN
min,18.000000,157.000000,47.000000,NaN,NaN,NaN,NaN,0.000000,NaN
25%,19.000000,172.000000,63.000000,NaN,NaN,NaN,NaN,4.275000,NaN
50%,21.000000,177.000000,72.000000,NaN,NaN,NaN,NaN,8.400000,NaN
75%,23.000000,182.000000,79.000000,NaN,NaN,NaN,NaN,12.100000,NaN


In [201]:
df.head()

,age_yrs,height_cm,weight_kg,duration_mins,elapsed_mins,sex,last_meal,units,over_limit
0,21,185,85,315,10,Male,Snack,19.7,Yes
1,34,187,95,240,5,Male,Lunch,10.2,No
2,23,200,101,60,10,Male,Snack,4.8,No
3,26,177,76,60,10,Male,Full,2.9,No
4,19,175,69,30,10,Male,Full,2.6,No


In [202]:
features=list(df.columns)

features.remove('over_limit')

features

['age_yrs',
 'height_cm',
 'weight_kg',
 'duration_mins',
 'elapsed_mins',
 'sex',
 'last_meal',
 'units']

In [203]:
numeric_features=['age_yrs','height_cm','weight_kg','duration_mins','elapsed_mins','units']
nominal_features=['sex','last_meal']

In [204]:
for feature in numeric_features:
    print(feature, df[feature].unique())

age_yrs [21 34 23 26 19 24 18 20 36 22 25 40 38 39 29 45 28]
height_cm [185 187 200 177 175 162 182 180 170 172 165 157 160 167]
weight_kg [ 85  95 101  76  69  57  82  63  79  66  73  88  84  71  75  72  47  60
  55  51  48  52  53  64  80]
duration_mins ['315' '240' '60' '30' '180' '90' '120' '325' '330' '?' '270' '150' '360'
 '300' '5' '435']
elapsed_mins ['10' '5' '15' '180' '120' '40' '?' '30' '60']
units [19.7 10.2  4.8  2.9  2.6 15.6  3.   5.  13.6 20.8 10.4  2.4  5.2 16.8
 16.1  7.2 10.   0.  12.6 13.2  9.   5.1  7.8  1.3 16.9  4.5  3.6 11.4
 12.   9.6  9.1 10.1 14.4  9.8  9.5  4.6 13.  16.6  1.2 15.8 12.4 31.2
 11.  15.   5.9]


In [205]:
for feature in nominal_features:
    print(feature, df[feature].unique())

sex ['Male' 'Female']
last_meal ['Snack' 'Lunch' 'Full' nan '?']


In [206]:
df.shape

(76, 9)

In [241]:
#df=(df[(df["duration_mins"] != '?') & (df["elapsed_mins"] != '?') & (df["last_meal"] != '?') & (df["last_meal"] != 'None')]).copy()

In [234]:
type(df)

pandas.core.frame.DataFrame

In [235]:
df.shape

(73, 9)

In [236]:
for feature in numeric_features:
    print(feature, df[feature].unique())

age_yrs [21 34 23 26 19 24 18 20 36 22 25 38 39 29 45 28]
height_cm [185 187 200 177 175 162 182 180 172 170 165 157 160 167]
weight_kg [ 85  95 101  76  69  57  82  63  79  66  73  88  84  71  72  47  60  55
  51  48  52  53  75  64  80]
duration_mins [315 240  60  30 180  90 120 325 330 270 150 360 300   5 435]
elapsed_mins [ 10   5  15 180 120  40  60  30]
units [19.7 10.2  4.8  2.9  2.6 15.6  3.   5.  13.6 20.8 10.4  2.4  5.2 16.8
 16.1  7.2 10.  12.6 13.2  9.   5.1  7.8  1.3 16.9  3.6 11.4 12.   9.6
  9.1 10.1 14.4  9.8  9.5  4.6 13.  16.6  1.2 15.8 12.4 31.2 11.  15.
  5.9]


In [237]:
for feature in nominal_features:
    print(feature, df[feature].unique())

sex ['Male' 'Female']
last_meal ['Snack' 'Lunch' 'Full' nan]


In [239]:
df["duration_mins"]=df["duration_mins"].astype('int64')
df["elapsed_mins"]=df["elapsed_mins"].astype('int64')

In [240]:
print(df.dtypes)

age_yrs            int64
height_cm          int64
weight_kg          int64
duration_mins      int64
elapsed_mins       int64
sex               object
last_meal         object
units            float64
over_limit        object
dtype: object


### Splitting time

#### don't forget to stratify!!

In [242]:
# Split off the test set: 20% of the dataset.
dev_df, test_df = train_test_split(df, train_size=0.8, random_state=2, stratify=df['over_limit'])

In [218]:
# It can be good to do this on a copy of the dataset (excluding the test set, of course)
copy_df = dev_df.copy()

### Data preprocess

In [219]:
# Create the preprocessor
preprocessor = ColumnTransformer([
        ("scaler", StandardScaler(), 
                numeric_features),
        ("nom", Pipeline([("imputer", SimpleImputer(missing_values=np.nan, strategy="most_frequent")), 
                          ("binarizer", OneHotEncoder(handle_unknown="ignore"))]), 
                nominal_features)],
        remainder="passthrough")

In [220]:
# Extract the features but leave as a DataFrame
dev_X = dev_df[features]
test_X = test_df[features]

# Target values, converted to a 1D numpy array
dev_y = dev_df["over_limit"].values
test_y = test_df["over_limit"].values

In [221]:
# Create a pipeline that combines the preprocessor with kNN
knn = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", KNeighborsRegressor())])

# Create a dictionary of hyperparameters for kNN
knn_param_grid = {"predictor__n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],}

# Create the grid search object which will find the best hyperparameter values based on validation error
knn_gs = GridSearchCV(knn, knn_param_grid, scoring="neg_mean_absolute_error", cv=10, refit=True)

# Run grid search by calling fit. It will also re-train on train+validation using the best parameters.
knn_gs.fit(dev_X, dev_y)

# Let's see how well we did
knn_gs.best_params_, knn_gs.best_score_

E:\info\III-1\AI\artificial_intelligence_2023_2024\venv\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "E:\info\III-1\AI\artificial_intelligence_2023_2024\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "E:\info\III-1\AI\artificial_intelligence_2023_2024\venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "E:\info\III-1\AI\artificial_intelligence_2023_2024\venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
     

({'predictor__n_neighbors': 1}, nan)